# Configuração, Importações e Carregamento

In [19]:
!pip install pyspark # Instala o PySpark no ambiente Colab

In [20]:
import pyspark #  Importa a biblioteca PySpark
import pandas as pd #  Importa a biblioteca Pandas
from pyspark.sql import SparkSession # Importa a classe SparkSession
from pyspark.sql.functions import col, count, when, regexp_extract # Importa Funções SQL: col (referência a coluna), count (agregação), when (lógica condicional SQL), e regexp_extract (extração por expressão regular)

In [21]:
from google.colab import drive # Importa o módulo drive do Google Colab
drive.mount('/content/drive') # Monta o Google Drive para acesso aos arquivos

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
spark = SparkSession.builder.getOrCreate() # Cria ou obtém uma instância do SparkSession

In [23]:
produtos = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/produtos.csv", header=True, inferSchema=True) # Lê o DataFrame produtos do Drive
clientes = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/clientes.csv", header=True, inferSchema=True) # Lê o DataFrame clientes do Drive
itens_pedidos = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/itens_pedido.csv", header=True, inferSchema=True) # Lê o DataFrame itens_pedidos do Drive
pedidos = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/pedidos.csv", header=True, inferSchema=True) # Lê o DataFrame pedidos do Drive

# Validação de Regras de Negócio (when)

In [24]:
itens_pedidos = itens_pedidos.withColumn("preco_valido", when(col("preco") > 0, "válido").otherwise("inválido")) # Cria a coluna preco_valido. Se o valor na coluna preco for maior que zero, marca como "válido", caso contrário, marca como "inválido"

itens_pedidos.select("id_pedido", "id_produto", "preco", "preco_valido").show() # Exibe as colunas relevantes, mostrando a nova coluna de validação
itens_pedidos.filter(col("preco_valido") == "inválido").show() # Filtra e exibe apenas os registros que falharam na validação de preço (preços menores ou iguais a zero)

+--------------------+--------------------+------+------------+
|           id_pedido|          id_produto| preco|preco_valido|
+--------------------+--------------------+------+------------+
|00010242fe8c5a6d1...|4244733e06e7ecb49...|  58.9|      válido|
|00018f77f2f0320c5...|e5f2d52b802189ee6...| 239.9|      válido|
|000229ec398224ef6...|c777355d18b72b67a...| 199.0|      válido|
|00024acbcdf0a6daa...|7634da152a4610f15...| 12.99|      válido|
|00042b26cf59d7ce6...|ac6c3623068f30de0...| 199.9|      válido|
|00048cc3ae777c65d...|ef92defde845ab845...|  21.9|      válido|
|00054e8431b9d7675...|8d4f2bb7e93e6710a...|  19.9|      válido|
|000576fe39319847c...|557d850972a7d6f79...| 810.0|      válido|
|0005a1a1728c9d785...|310ae3c140ff94b03...|145.95|      válido|
|0005f50442cb953dc...|4535b0e1091c278df...| 53.99|      válido|
|00061f2a7bc09da83...|d63c1011f49d98b97...| 59.99|      válido|
|00063b381e2406b52...|f177554ea93259a5b...|  45.0|      válido|
|0006ec9db01a64e59...|99a4788cb24856965.

In [25]:
status_permitidos = ['created','approved','delivered','shipped','canceled','invoiced','processing'] # Define uma lista de valores válidos para o status do pedido

pedidos = pedidos.withColumn("status_valido", when(col("status_pedido").isin(status_permitidos), "válido").otherwise("Inválido")) # Cria a coluna status_valido. Se o valor na coluna status_pedido estiver contido na lista status_permitidos (.isin()), marca como "válido", caso contrário, marca como "Inválido"

pedidos.select("id_pedido", "status_pedido", "status_valido").show() # Exibe os resultados da validação de status

pedidos.filter(col("status_valido") == "Inválido").show() # Filtra e exibe apenas os registros que falharam na validação de status (status desconhecido ou não permitido)

+--------------------+-------------+-------------+
|           id_pedido|status_pedido|status_valido|
+--------------------+-------------+-------------+
|e481f51cbdc54678b...|    delivered|       válido|
|53cdb2fc8bc7dce0b...|    delivered|       válido|
|47770eb9100c2d0c4...|    delivered|       válido|
|949d5b44dbf5de918...|    delivered|       válido|
|ad21c59c0840e6cb8...|    delivered|       válido|
|a4591c265e18cb1dc...|    delivered|       válido|
|136cce7faa42fdb2c...|     invoiced|       válido|
|6514b8ad8028c9f2c...|    delivered|       válido|
|76c6e866289321a7c...|    delivered|       válido|
|e69bfb5eb88e0ed6a...|    delivered|       válido|
|e6ce16cb79ec1d90b...|    delivered|       válido|
|34513ce0c4fab462a...|    delivered|       válido|
|82566a660a982b15f...|    delivered|       válido|
|5ff96c15d0b717ac6...|    delivered|       válido|
|432aaf21d85167c2c...|    delivered|       válido|
|dcb36b511fcac050b...|    delivered|       válido|
|403b97836b0c04a62...|    deliv

# Análise de Nulos (Nulls)

In [26]:
for coluna in clientes.columns:
  clientes.select(count(col(coluna)).alias(coluna)).show() # Loop para iterar sobre cada coluna em clientes. Para cada coluna, calcula a contagem de valores não-nulos e exibe o resultado

for coluna in clientes.columns:
  clientes.select(count(when(col(coluna).isNull(), coluna)).alias(coluna)).show() # Loop para iterar sobre cada coluna. Usa a função when() para marcar NULLs, e o count() subsequente conta apenas os valores nulos em cada coluna, exibindo o resultado coluna por coluna

+----------+
|id_cliente|
+----------+
|     99441|
+----------+

+----------------+
|id_unico_cliente|
+----------------+
|           99441|
+----------------+

+-----------+
|cep_cliente|
+-----------+
|      99441|
+-----------+

+--------------+
|cidade_cliente|
+--------------+
|         99441|
+--------------+

+--------------+
|estado_cliente|
+--------------+
|         99441|
+--------------+

+----------+
|id_cliente|
+----------+
|         0|
+----------+

+----------------+
|id_unico_cliente|
+----------------+
|               0|
+----------------+

+-----------+
|cep_cliente|
+-----------+
|          0|
+-----------+

+--------------+
|cidade_cliente|
+--------------+
|             0|
+--------------+

+--------------+
|estado_cliente|
+--------------+
|             0|
+--------------+



In [27]:
n_nulls_pedidos_df = pedidos.select([count(when(col(c).isNull(), c)).alias(c) for c in pedidos.columns]) # Usa uma list comprehension para criar uma lista de expressões de contagem de nulos para todas as colunas de pedidos. O resultado é um único DataFrame com uma linha, onde cada coluna mostra a contagem de nulos naquela coluna
n_nulls_pedidos_df.show() # Exibe o DataFrame de resumo da contagem de nulos

+---------+----------+-------------+------------------+---------------------+-------------------------+--------------------+---------------------+-------------+
|id_pedido|id_cliente|status_pedido|data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|status_valido|
+---------+----------+-------------+------------------+---------------------+-------------------------+--------------------+---------------------+-------------+
|        0|         0|            0|                 0|                  160|                     1783|                2965|                    0|            0|
+---------+----------+-------------+------------------+---------------------+-------------------------+--------------------+---------------------+-------------+



# Análise de Integridade Referencial (left_anti join)

In [28]:
pedidos_integridade_df = pedidos.join(clientes, pedidos.id_cliente == clientes.id_cliente, "left_anti") # Realiza um left_anti join entre pedidos (esquerda) e clientes (direita) na chave id_cliente. O resultado são apenas os pedidos que contêm um id_cliente que não existe no DataFrame clientes
pedidos_integridade_df.select("id_pedido", "id_cliente").show() # Exibe os IDs dos pedidos com problemas de integridade

+---------+----------+
|id_pedido|id_cliente|
+---------+----------+
+---------+----------+



In [29]:
itens_integridade_df = itens_pedidos.join(produtos, itens_pedidos.id_produto == produtos.id_produto, "left_anti") # Realiza um left_anti join entre itens_pedidos e produtos. O resultado são apenas os itens de pedido que contêm um id_produto que não existe no DataFrame produtos
itens_integridade_df.select("id_pedido", "id_produto").show() # Exibe os IDs de item de pedido com produtos inexistentes

+---------+----------+
|id_pedido|id_produto|
+---------+----------+
+---------+----------+



# Validação de Formato (rlike e Regex)

In [30]:
clientes_validacao_df = clientes.withColumn("cep_valido", when(col("cep_cliente").rlike(r"^\d{5}$"), "válido").otherwise("Inválido"))  # Cria a coluna cep_valido. O rlike verifica se o cep_cliente corresponde ao padrão regex: * ^: Início da string. * \d{5}: Exatamente 5 dígitos numéricos. * $: Fim da string
clientes_validacao_df.select("id_cliente", "cep_cliente", "cep_valido").show() # Exibe os resultados da validação de CEP

+--------------------+-----------+----------+
|          id_cliente|cep_cliente|cep_valido|
+--------------------+-----------+----------+
|06b8999e2fba1a1fb...|      14409|    válido|
|18955e83d337fd6b2...|       9790|  Inválido|
|4e7b3e00288586ebd...|       1151|  Inválido|
|b2b6027bc5c5109e5...|       8775|  Inválido|
|4f2d8ab171c80ec83...|      13056|    válido|
|879864dab9bc30475...|      89254|    válido|
|fd826e7cf63160e53...|       4534|  Inválido|
|5e274e7a0c3809e14...|      35182|    válido|
|5adf08e34b2e99398...|      81560|    válido|
|4b7139f34592b3a31...|      30575|    válido|
|9fb35e4ed6f0a14a4...|      39400|    válido|
|5aa9e4fdd4dfd2095...|      20231|    válido|
|b2d1536598b73a9ab...|      18682|    válido|
|eabebad39a88bb6f5...|       5704|  Inválido|
|1f1c7bf1c9b041b29...|      95110|    válido|
|206f3129c0e4d7d0b...|      13412|    válido|
|a7c125a0a07b75146...|      22750|    válido|
|c5c61596a3b6bd0ce...|       7124|  Inválido|
|9b8ce803689b3562d...|       5416|

In [31]:
produtos_validacao_df = produtos.withColumn("id_produto_valido", when(col("id_produto").rlike(r"^[a-f0-9]{32}$"), "válido").otherwise("Inválido")) # Cria a coluna id_produto_valido. O rlike verifica se o id_produto corresponde ao padrão de um hash: * ^: Início da string. * [a-f0-9]: Qualquer caractere que seja uma letra de 'a' a 'f' ou um dígito de '0' a '9'. * {32}: O padrão deve ocorrer exatamente 32 vezes. * $: Fim da string
produtos_validacao_df.select("id_produto", "id_produto_valido").show() # Exibe os resultados da validação de formato do ID de produto

+--------------------+-----------------+
|          id_produto|id_produto_valido|
+--------------------+-----------------+
|1e9e8ef04dbcff454...|           válido|
|3aa071139cb16b67c...|           válido|
|96bd76ec8810374ed...|           válido|
|cef67bcfe19066a93...|           válido|
|9dc1a7de274444849...|           válido|
|41d3672d4792049fa...|           válido|
|732bd381ad09e530f...|           válido|
|2548af3e6e77a690c...|           válido|
|37cc742be07708b53...|           válido|
|8c92109888e8cdf9d...|           válido|
|14aa47b7fe5c25522...|           válido|
|03b63c5fc16691530...|           válido|
|cf55509ea8edaaac1...|           válido|
|7bb6f29c2be577161...|           válido|
|eb31436580a610f20...|           válido|
|3bb7f144022e67327...|           válido|
|6a2fb4dd53d2cdb88...|           válido|
|a1b71017a84f92fd8...|           válido|
|a0736b92e52f6cead...|           válido|
|f53103a77d9cf245e...|           válido|
+--------------------+-----------------+
only showing top

In [32]:
if "email_cliente" in clientes.columns: #Bloco if para garantir que o código só tente validar o e-mail se a coluna email_cliente realmente existir no DataFrame clientes
  clientes_verifica_email_df = clientes.withColumn("email_valido", when(col("email_cliente").contains("@"), "válido").otherwise("Inválido")) # Cria a coluna email_valido. Usa o método .contains("@") para uma validação simples: se a string contiver o caractere @, é marcado como "válido"
  clientes_verifica_email_df.select("id_cliente", "email_cliente", "email_valido").show() # Exibe os resultados da validação simples de e-mail